In [1]:
import tensorflow as tf
import datetime
import pickle
from enum import Enum
# from tensorflow.python.keras import backend as K

In [2]:
class unPickType(Enum):
    TRAIN_X = 1
    TRAIN_Y = 2
    TEST_X = 3
    TEST_Y = 4
    VAL_X = 5
    VAL_Y = 6

def unPick(type):
    f_path = "Pickles/"
    x = type.value%2 == 1
    if type.value in [1, 2]:
        f_path += f"train_{'x' if x else 'y'}"

    elif type.value in [3, 4]:
        f_path += f"test_{'x' if x else 'y'}"

    elif type.value in [5, 6]:
        f_path += f"val_{'x' if x else 'y'}"

    else:
        raise ValueError

    f_path += ".txt"

    file = open(f_path, "rb")
    res = pickle.load(file)
    file.close()
    return res

train_x = unPick(unPickType.TRAIN_X)
train_y = unPick(unPickType.TRAIN_Y)
test_x = unPick(unPickType.TEST_X)
test_y = unPick(unPickType.TEST_Y)
val_x = unPick(unPickType.VAL_X)
val_y = unPick(unPickType.VAL_Y)

In [3]:
# tf_config = tf.ConfigProto(allow_soft_placement=False)
# tf_config.gpu_options.allow_growth = True
# s = tf.Session(config=tf_config)
# K.set_session(s)

In [4]:
norm_init=tf.keras.initializers.RandomNormal(mean=0., stddev=0.1, seed=42)

In [5]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
callback = tf.keras.callbacks.EarlyStopping(monitor='sparse_categorical_accuracy', patience=5, mode="max")

In [6]:
def get_basic_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(12, input_shape=(train_x.shape[1:]), kernel_initializer=norm_init),
        tf.keras.layers.SimpleRNN(256, return_sequences=True, kernel_initializer=norm_init),
        tf.keras.layers.SimpleRNN(256, kernel_initializer=norm_init),
        tf.keras.layers.Dense(9, kernel_initializer=norm_init)
    ])

    return model

model = get_basic_model()

In [7]:
model.compile(optimizer='RMSProp',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['sparse_categorical_accuracy'])

In [8]:
EPOCHS = 100
BATCH_SIZE = 5

In [9]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
with tf.device('/GPU:0'):
    history = model.fit(
        train_x, train_y,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(val_x, val_y),
        verbose=1,
        callbacks=[tensorboard_callback],
        use_multiprocessing=True,
        workers=8
    )

Epoch 1/100
1395/1395 [==============================] - 41s 27ms/step - loss: 14.0423 - sparse_categorical_accuracy: 0.1195 - val_loss: 14.3235 - val_sparse_categorical_accuracy: 0.1113
Epoch 2/100
1395/1395 [==============================] - 35s 25ms/step - loss: 14.1715 - sparse_categorical_accuracy: 0.1208 - val_loss: 14.3235 - val_sparse_categorical_accuracy: 0.1113
Epoch 3/100
1395/1395 [==============================] - 38s 27ms/step - loss: 14.1715 - sparse_categorical_accuracy: 0.1208 - val_loss: 14.3235 - val_sparse_categorical_accuracy: 0.1113
Epoch 4/100
1395/1395 [==============================] - 37s 27ms/step - loss: 14.1715 - sparse_categorical_accuracy: 0.1208 - val_loss: 14.3235 - val_sparse_categorical_accuracy: 0.1113
Epoch 5/100
1395/1395 [==============================] - 37s 26ms/step - loss: 14.1715 - sparse_categorical_accuracy: 0.1208 - val_loss: 14.3235 - val_sparse_categorical_accuracy: 0.1113
Epoch 6/100
1395/1395 [==============================] - 37s 26ms

In [11]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

Launching TensorBoard...